In [1]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests

url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "format": "json",  # Ensure the response is in JSON format
    "query.term": "covid",  # Example search term
    "pageSize": 10,  # Number of results per page
}

response = requests.get(url, params=params)
data = response.json()

# Print trial titles
for trial in data["studies"]:
    print(trial["protocolSection"]["identificationModule"]["officialTitle"])


A Randomized, Double-Blind, Placebo-Controlled, Phase 1 Study to Evaluate the Safety, Pharmacokinetics, and Pharmacodynamics of Single Ascending Doses of ABBV-47D11 and ABBV-2B04 as Monotherapy or Combination Therapy in Adults With COVID-19
Personalised Electronic Record Supported Optimisation When Alone for Patients With Hypertension- Pilot Study for Remote Medical Management of Hypertension During the COVID-19 Pandemic
COVID-19 Ring-based Prevention Trial With Lopinavir/Ritonavir
The Mental Health Status of the General Population During the COVID-19 Pandemic and Its Association With Adherence to Government-initiated Non-pharmacological Epidemiological Interventions (NPI's)
PROSPECTIVE OPEN LABEL CLINICAL TRIAL TO ADMINISTER A BOOSTER DOSE OF PFIZER/BIONTECH OR MODERNA COVID-19 VACCINE IN HIGH-RISK INDIVIDUALS
A Randomised Controlled Trial to Assess the Immunogenicity, Safety and Reactogenicity of Standard Dose Versus Fractional Doses of COVID-19 Vaccine (Pfizer-BioNTech) Given as a B

In [15]:

import requests

# Base URL
url = "https://clinicaltrials.gov/api/v2/studies"

# Simple search parameters

params = {
    "format": "json",
    "query.cond": "breast cancer",
    "query.locn": "Boston",  # Location search
    "filter.advanced": "AREA[MinimumAge]RANGE[MIN, 65 years]",  # Age criteria
    "fields": ",".join(["NCTId", "BriefTitle", "EligibilityCriteria", "LocationFacility", "LocationCity", "OverallStatus"]),
    "pageSize": 10,
}

# Make the request with proper URL encoding
response = requests.get(url, params=params)

# Check if the response is successful before parsing JSON
if response.status_code == 200:
    try:
        data = response.json()
        print(f"Total studies found: {data.get('totalCount', 'N/A')}")
        for i, trial in enumerate(data.get("studies", []), 1):
            print(f"\n{i}. {trial['protocolSection']['identificationModule']['nctId']}")
            print(f"   {trial['protocolSection']['identificationModule']['briefTitle']}")
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        print(f"Response content: {response.text[:200]}...")  # Print first 200 chars
else:
    print(f"Error: {response.status_code}")
    print(f"Response: {response.text[:200]}...")

Total studies found: N/A

1. NCT00829166
   A Study of Trastuzumab Emtansine Versus Capecitabine + Lapatinib in Participants With HER2-positive Locally Advanced or Metastatic Breast Cancer

2. NCT01337765
   Safety, Pharmacokinetics and Pharmacodynamics of BEZ235 Plus MEK162 in Selected Advanced Solid Tumor Patients

3. NCT00003565
   Docetaxel in Treating Patients With Solid Tumors

4. NCT05579366
   Rinatabart Sesutecan (Rina-S) for Advanced Solid Tumors (GCT1184-01/ PRO1184-001)

5. NCT03845166
   A Study of XL092 as Single-Agent and Combination Therapy in Subjects With Solid Tumors

6. NCT01427166
   Non-Invasive Imaging for Cording in Breast Cancer Patients

7. NCT05696626
   Evaluation of Lasofoxifene Combined With Abemaciclib Compared With Fulvestrant Combined With Abemaciclib in Locally Advanced or Metastatic ER+/HER2- Breast Cancer With an ESR1 Mutation

8. NCT00079326
   Trastuzumab and Ixabepilone in Treating Women With HER2-Positive Metastatic Breast Cancer

9. NCT04140526


In [16]:
import pandas as pd
import requests

# Base URL
url = "https://clinicaltrials.gov/api/v2/studies"

# Function to extract key information from a trial
def process_trial(trial):
    protocol = trial["protocolSection"]
    id_module = protocol["identificationModule"]
    status_module = protocol.get("statusModule", {})
    design_module = protocol.get("designModule", {})
    eligibility = protocol.get("eligibilityModule", {})
    
    return {
        "nct_id": id_module.get("nctId"),
        "title": id_module.get("briefTitle"),
        "status": status_module.get("overallStatus"),
        "phase": design_module.get("phases", [None])[0] if design_module.get("phases") else None,
        "enrollment": design_module.get("enrollmentInfo", {}).get("count"),
        "gender": eligibility.get("sex"),
        "min_age": eligibility.get("minimumAge"),
        "max_age": eligibility.get("maximumAge"),
        "has_results": trial.get("hasResults", False),
    }

# Update parameters to include all fields needed by process_trial
params = {
    "format": "json",
    "query.cond": "breast cancer",
    "query.locn": "Boston",
    "filter.advanced": "AREA[MinimumAge]RANGE[MIN, 65 years]",
    "fields": ",".join([
        "NCTId", 
        "BriefTitle", 
        "OverallStatus",
        "Phase", 
        "EnrollmentInfo",
        "Sex",
        "MinimumAge",
        "MaximumAge",
        "hasResults",
    ]),
    "pageSize": 10,
}

# Make the request
response = requests.get(url, params=params)

# Process the response
if response.status_code == 200:
    data = response.json()
    print(f"Total studies found: {data.get('totalCount', 'N/A')}")
    
    # Process all trials into structured dictionaries
    results = [process_trial(trial) for trial in data.get("studies", [])]
    
    # Display results (as a table using pandas for better formatting)
    if results:
        df = pd.DataFrame(results)
        print(df)
    else:
        print("No studies found matching criteria")
else:
    print(f"Error: {response.status_code}")
    print(f"Response: {response.text[:200]}...")

Total studies found: N/A
        nct_id                                              title  \
0  NCT00829166  A Study of Trastuzumab Emtansine Versus Capeci...   
1  NCT01337765  Safety, Pharmacokinetics and Pharmacodynamics ...   
2  NCT00003565   Docetaxel in Treating Patients With Solid Tumors   
3  NCT05579366  Rinatabart Sesutecan (Rina-S) for Advanced Sol...   
4  NCT03845166  A Study of XL092 as Single-Agent and Combinati...   
5  NCT01427166  Non-Invasive Imaging for Cording in Breast Can...   
6  NCT05696626  Evaluation of Lasofoxifene Combined With Abema...   
7  NCT00079326  Trastuzumab and Ixabepilone in Treating Women ...   
8  NCT04140526  Safety, PK and Efficacy of ONC-392 in Monother...   
9  NCT01254526  Study of the Safety and Pharmacology of GDC-09...   

                  status   phase  enrollment  gender   min_age max_age  \
0              COMPLETED  PHASE3         991     ALL  18 Years    None   
1              COMPLETED  PHASE1          29     ALL  18 Years    N

In [22]:
pip install openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.3 MB/s eta 0:00:00
  Using cached types_requests-2.32.0.20250328-py3-none-any.whl.metadata (2.3 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.5 MB/s eta 0:00:0000:01
Using cached types_requests-2.32.0.20250328-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 15.1 MB/s eta 0:00:00
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 6.1 MB/s eta 0:00:00
 

In [27]:
# Step 2: Load the .env file
import os

from dotenv import load_dotenv

# This loads variables from .env into environment
load_dotenv()  

# Verify it's loaded
api_key = os.environ.get("OPENAI_API_KEY")

In [28]:
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

# For Jupyter/IPython notebooks, use the async version directly:
async def run_agent():
    result = await Runner.run(
        starting_agent=agent, 
        input="Write a haiku about recursion in programming.",
    )
    return result

# This works in Jupyter
result = await run_agent()
print(result.final_output)

Code within code loops,  
Infinite self-reference,  
Logic twists and turns.


In [11]:
# Function to extract key information from a trial
def process_trial(trial):
    protocol = trial["protocolSection"]
    id_module = protocol["identificationModule"]
    status_module = protocol.get("statusModule", {})
    design_module = protocol.get("designModule", {})
    eligibility = protocol.get("eligibilityModule", {})
    
    return {
        "nct_id": id_module.get("nctId"),
        "title": id_module.get("briefTitle"),
        "status": status_module.get("overallStatus"),
        "phase": design_module.get("phases", [None])[0],
        "enrollment": design_module.get("enrollmentInfo", {}).get("count"),
        "gender": eligibility.get("sex"),
        "min_age": eligibility.get("minimumAge"),
        "max_age": eligibility.get("maximumAge"),
        "has_results": trial.get("hasResults", False),
    }

# Process results
results = [process_trial(trial) for trial in data.get("studies", [])]

In [42]:
await main()

# For Jupyter compatibility
try:
    loop = asyncio.get_running_loop()
    # In Jupyter, use top-level await
    await main()
except RuntimeError:
    # In regular Python script, use asyncio.run()
    asyncio.run(main())


import requests
from agents import Agent, ModelSettings, function_tool


@function_tool
def query_ctg(
    query: str,
    phase: str | None = None,
    recruitmentStatus: str | None = None,
    locationState: str | None = None,
) -> str:
    # … your existing code …
    r = requests.get("https://clinicaltrials.gov/api/v2/studies", params=params, timeout=30)
    r.raise_for_status()
    return r.text

agent = Agent(
    name="TrialFinder",                              
    instructions=(
      "Turn the user's request into precise ClinicalTrials.gov parameters "
      "using enum values; retry with different parameters if the API responds with an error."
    ),
    model="gpt-4o",                                   # ← use `model`, not `model_config`
    model_settings=ModelSettings(temperature=0.0),    # ← optional tuning
    tools=[query_ctg],                                # your @function_tool
)

async def main():
    result = await Runner.run(agent, "Find HER2+ breast cancer trials recruiting in California, Phase 2")
    print(result.final_output)

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())


Here are some cancer trials recruiting in California, Phase 2:

1. **Rinatabart Sesutecan (Rina-S) for Advanced Solid Tumors**
   - **NCT ID:** [NCT05579366](https://clinicaltrials.gov/ct2/show/NCT05579366)
   - **Status:** Recruiting
   - **Phase:** Phase 1/Phase 2
   - **Enrollment:** 404 (Estimated)
   - **Eligibility:** All sexes, 18 years and older

2. **Safety, PK and Efficacy of ONC-392 in Monotherapy and in Combination of Anti-PD-1 in Advanced Solid Tumors and NSCLC**
   - **NCT ID:** [NCT04140526](https://clinicaltrials.gov/ct2/show/NCT04140526)
   - **Status:** Recruiting
   - **Phase:** Phase 1/Phase 2
   - **Enrollment:** 733 (Estimated)
   - **Eligibility:** All sexes, 18 years and older

These trials are currently recruiting participants. You can click on the NCT ID to get more details about each study.
Here are some cancer trials recruiting in California, Phase 2:

1. **Rinatabart Sesutecan (Rina-S) for Advanced Solid Tumors**
   - **NCT ID:** [NCT05579366](https://clini

RuntimeError: asyncio.run() cannot be called from a running event loop